1 MILLION, 10 MILLION, 100 MILLION TRIALS WITH PARALLELIZATION

In [ ]:
import numpy as np
from collections import Counter, defaultdict
import concurrent.futures

# Define ranks and suits
ranks = np.arange(13)  # Map ranks to integers 0-12 (2 to Ace)
suits = np.arange(4)   # Map suits to integers 0-3 (hearts, diamonds, clubs, spades)

# Create a deck with two identical sets of 52 cards
deck = np.array([(rank, suit) for rank in ranks for suit in suits] * 2, dtype=[('rank', 'i4'), ('suit', 'i4')])

def print_deck(deck):
    """Print the entire deck to show all cards in both identical sets."""
    for card in deck:
        rank = card['rank']
        suit = card['suit']
        suit_name = ["Hearts", "Diamonds", "Clubs", "Spades"][suit]
        rank_name = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"][rank]
        print(f"{rank_name} of {suit_name}", end=", ")
    print("\n")

def print_hand(hand):
    """Print the hand in a readable 'rank of suit' format."""
    hand_cards = []
    for card in hand:
        rank = card['rank']
        suit = card['suit']
        suit_name = ["Hearts", "Diamonds", "Clubs", "Spades"][suit]
        rank_name = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"][rank]
        hand_cards.append(f"{rank_name} of {suit_name}")
    print("Hand:", ", ".join(hand_cards))

def deal_hand(deck):
    """
    Shuffle the deck and select five unique cards.
    """
    shuffled_deck = np.random.permutation(deck)
    hand_indices = np.random.choice(len(shuffled_deck), 5, replace=False)
    hand = shuffled_deck[hand_indices]
    sorted_hand = np.sort(hand, order='rank')
    return shuffled_deck, sorted_hand

def check_four_of_a_kind(hand):
    """Check if a hand has a four-of-a-kind and determine its case."""
    ranks = hand['rank']
    suits = hand['suit']

    rank_counts = Counter(ranks)

    # Check for four of a kind
    four_of_a_kind_rank = [rank for rank, count in rank_counts.items() if count == 4]
    if not four_of_a_kind_rank:
        return None  # No four-of-a-kind found

    # Get suits of the four-of-a-kind rank cards
    four_kind_suits = [suit for rank, suit in zip(ranks, suits) if rank == four_of_a_kind_rank[0]]
    suit_counter = Counter(four_kind_suits)

    # Determine case based on suit distribution
    unique_suits = len(suit_counter)
    if unique_suits == 4:
        return "Case 0"
    elif unique_suits == 3 and 2 in suit_counter.values():
        return "Case 1"
    elif unique_suits == 2 and all(count == 2 for count in suit_counter.values()):
        return "Case 2"
    else:
        return None

def run_simulation(num_trials, worker_id=None):
    """
    Run simulation with enhanced random seed handling.
    Counts Four of a Kind cases with batching.
    """
    if worker_id is not None:
        # Create unique seed using large prime numbers and worker id
        base_seed = 104729  # Large prime
        multiplier = 15485863  # Another large prime
        worker_seed = base_seed + (worker_id * multiplier)
        np.random.seed(worker_seed)

    counts = Counter()
    local_examples = defaultdict(list)  # Store examples for each case

    # Process hands in smaller batches for better progress tracking
    batch_size = 100_000
    num_batches = num_trials // batch_size + (1 if num_trials % batch_size else 0)

    for batch in range(num_batches):
        current_batch_size = min(batch_size, num_trials - batch * batch_size)
        batch_counts = Counter()

        for _ in range(current_batch_size):
            _, hand = deal_hand(deck)
            case = check_four_of_a_kind(hand)
            if case:
                batch_counts[case] += 1
                # Collect up to 10 examples for each case
                if len(local_examples[case]) < 10:
                    local_examples[case].append(hand.copy())

        # Update total counts
        counts.update(batch_counts)

        if worker_id is not None:
            print(f"Worker {worker_id}: Completed batch {batch + 1}/{num_batches}, "
                  f"found Case 0: {counts['Case 0']}, "
                  f"Case 1: {counts['Case 1']}, "
                  f"Case 2: {counts['Case 2']} so far")

    return counts, local_examples

def run_simulation_parallel(trials, num_workers=16):
    """
    Run parallel simulation with improved worker management and progress tracking.
    """
    chunk_size = trials // num_workers
    remaining = trials % num_workers
    total_counts = Counter()
    all_examples = defaultdict(list)

    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
        # Distribute work with adjusted chunks
        futures = []
        for i in range(num_workers):
            # Last worker gets remaining hands if any
            worker_trials = chunk_size + (remaining if i == num_workers-1 else 0)
            futures.append(executor.submit(run_simulation, worker_trials, worker_id=i))

        # Track progress and collect results
        completed = 0
        results = []
        for future in concurrent.futures.as_completed(futures):
            completed += 1
            counts, examples = future.result()
            results.append((counts, examples))

    # Combine all results
    for counts, examples in results:
        total_counts.update(counts)
        # Merge examples up to 10 per case
        for case, hands in examples.items():
            if len(all_examples[case]) < 10:
                all_examples[case].extend(hands[:10 - len(all_examples[case])])

    # Print final summary
    print("\nTotal examples collected:")
    for case in ["Case 0", "Case 1", "Case 2"]:
        print(f"{case}: {len(all_examples[case])} examples")

    return total_counts, all_examples

def calculate_total_four_of_a_kind_probability(counts, total_hands):
    """Calculate the total probability of getting any four-of-a-kind."""
    total_four_of_a_kind = sum(counts[case] for case in ["Case 0", "Case 1", "Case 2"])
    return total_four_of_a_kind / total_hands

def main():
    # Print the initial unshuffled deck
    print("Initial deck:")
    print_deck(deck)

    # Part 1: Run 100 trials, shuffle each time, and print the shuffled deck and dealt hand
    print("\nRunning 100 trials:")
    example_hands_100 = {"Case 0": [], "Case 1": [], "Case 2": []}
    for trial in range(100):
        shuffled_deck, hand = deal_hand(deck)  # Shuffle and deal a hand
        print(f"\nTrial {trial + 1}: Shuffled deck")
        print_deck(shuffled_deck)  # Print the entire shuffled deck after shuffling

        # Print the first 5 cards as the dealt hand
        print_hand(hand)

        # Check for a four-of-a-kind in the dealt hand
        case = check_four_of_a_kind(hand)
        print(f"Four-of-a-kind Case: {case}")

        # Collect examples during the initial 100 trials if needed
        if case in example_hands_100 and len(example_hands_100[case]) < 10:
            example_hands_100[case].append(hand)

    # Part 2: Run full simulation for larger trial counts and calculate total four-of-a-kind probability
    trial_counts = [1_00, 1_000, 10_000, 100_000, 1_000_000, 10_000_000, 100_000_000]
    for trials in trial_counts:
        print(f"\nRunning parallel simulation with {trials} trials...")
        counts, example_hands = run_simulation_parallel(trials)

        # Calculate total probability
        total_probability = calculate_total_four_of_a_kind_probability(counts, trials)

        print(f"Results for {trials} trials:")
        print(f"Case 0 (all different suits): {counts['Case 0']}")
        print(f"Case 1 (two same suit): {counts['Case 1']}")
        print(f"Case 2 (two pairs same suit): {counts['Case 2']}")
        print(f"Total probability: {total_probability:.8f}")

        # Print examples for 1,000,000 trial run
        if trials == 1_000_000:
            print("\nExample hands from 1,000,000 trials:")
            for case in ["Case 0", "Case 1", "Case 2"]:
                print(f"\n{case} examples:")
                for hand in example_hands[case]:
                    print_hand(hand)

if __name__ == "__main__":
    main()


Initial deck:
2 of Hearts, 2 of Diamonds, 2 of Clubs, 2 of Spades, 3 of Hearts, 3 of Diamonds, 3 of Clubs, 3 of Spades, 4 of Hearts, 4 of Diamonds, 4 of Clubs, 4 of Spades, 5 of Hearts, 5 of Diamonds, 5 of Clubs, 5 of Spades, 6 of Hearts, 6 of Diamonds, 6 of Clubs, 6 of Spades, 7 of Hearts, 7 of Diamonds, 7 of Clubs, 7 of Spades, 8 of Hearts, 8 of Diamonds, 8 of Clubs, 8 of Spades, 9 of Hearts, 9 of Diamonds, 9 of Clubs, 9 of Spades, 10 of Hearts, 10 of Diamonds, 10 of Clubs, 10 of Spades, J of Hearts, J of Diamonds, J of Clubs, J of Spades, Q of Hearts, Q of Diamonds, Q of Clubs, Q of Spades, K of Hearts, K of Diamonds, K of Clubs, K of Spades, A of Hearts, A of Diamonds, A of Clubs, A of Spades, 2 of Hearts, 2 of Diamonds, 2 of Clubs, 2 of Spades, 3 of Hearts, 3 of Diamonds, 3 of Clubs, 3 of Spades, 4 of Hearts, 4 of Diamonds, 4 of Clubs, 4 of Spades, 5 of Hearts, 5 of Diamonds, 5 of Clubs, 5 of Spades, 6 of Hearts, 6 of Diamonds, 6 of Clubs, 6 of Spades, 7 of Hearts, 7 of Diamonds, 